In [10]:
# !pip install sentence_transformers
# !pip install ftfy
# !pip install unidecode

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
from sentence_transformers import util
import torch
import string

c:\Users\Lenovo\anaconda3\envs\ChromaDB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('C:\TFG\DataProcessed\independencia_df_ca.csv')

In [3]:
def preprocess_text_independence_ca(tweet):
    '''
    la función preprocess_text_independence recibe un tweet y realiza una serie de transformaciones
    parámetros:
    - tweet: texto del tweet en catalán
    retorna:
    - text: texto preprocesado en catalán
    '''
    text = re.sub(r'@\w+|#\w+', '', tweet)
    # se eliminan los signos de puntuación de cada tweet
    text = "".join([char for char in text if char not in string.punctuation])
    emojis = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticonos
                        u"\U0001F300-\U0001F5FF"  # símbolos y emoticonos
                        u"\U0001F680-\U0001F6FF"  # emoticonos suplementarios
                        u"\U0001F900-\U0001F9FF"  # emoticonos ideográficos
                        u"\U0001FA00-\U0001FA6F"  # emoticonos de símbolos diversos
                        u"\U0001FA70-\U0001FAFF"  # emoticonos de transporte y objetos
                        u"\U0001F000-\U0001F0FF"  
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    text = emojis.sub('', text)
    # se eliminan las URL que empiezan por http
    text = re.sub(r'http\S+', '', text)
    # se eliminan las URL que empiezan por www
    text = re.sub(r'www\S+', '', text)
    # se eliminan las URL que empiezan por https
    text = re.sub(r'https\S+', '', text)
    return text

In [4]:
# se aplica la función preprocess_text_independence al dataset en español
df['Texto_preprocesado'] = df['Texto'].apply(lambda x: preprocess_text_independence_ca(x))

In [5]:
df

,Id,Texto,Sentimiento,Texto_preprocesado
0,1.102852e+34,En @fgarrobo ha fet més per l’independentisme ...,Negativo,En ha fet més per l’independentisme i la unit...
1,1.098893e+18,Divendres 22F acte Cloenda campanya de les can...,Positivo,Divendres 22F acte Cloenda campanya de les can...
2,1.099032e+34,Puigdemont porta a la Defensora del Poble euro...,Neutral,Puigdemont porta a la Defensora del Poble euro...
3,1.102519e+32,"José Antonio Nieto, ex-secretari d'Estat de Se...",Neutral,José Antonio Nieto exsecretari dEstat de Segur...
4,1.093082e+18,Es passen dies denunciant l'autocar amb que va...,Positivo,Es passen dies denunciant lautocar amb que van...
...,...,...,...,...
10043,1.098828e+34,@JoanTarda @I_CSC Això és un sindicat...i tu e...,Negativo,Això és un sindicati tu ets de esquerres Est...
10044,1.097865e+18,@miqueliceta @PSOE @socialistes_cat Deixi de p...,Positivo,Deixi de prendre els catalans per imbècils ...
10045,1.097985e+18,L'@annasaliente ens parla avui de filosofia i ...,Positivo,L ens parla avui de filosofia i a la primera ...
10046,1.103684e+18,MHP @QuimTorraiPla @JuntsXCat @Pdemocratacat @...,Positivo,MHP Si no heu complert amb el mandat del ...


In [6]:
df = df[['Texto_preprocesado', 'Sentimiento']]

In [15]:
# guardar df preprocesado para retrieval
df.to_csv('C:\TFG\DataProcessed\independencia_df_ca_retrieval.csv', index=False)

In [45]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

Is CUDA available: True
CUDA version: 12.1
cuDNN version: 8801


### Version CHROMADB y LANGCCHAIN

In [7]:
import chromadb

client = chromadb.PersistentClient(path="./database_ca")

In [8]:
collection = client.get_or_create_collection(name="tweets_ca")

In [9]:
documents = []
metadatas = []
ids = []
for j, item in enumerate(df[['Sentimiento', 'Texto_preprocesado']].values):
    ids.append(str(j))
    documents.append(item[1])
    metadatas.append({"title": item[0]})

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")

c:\Users\Lenovo\anaconda3\envs\ChromaDB\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Lenovo\anaconda3\envs\ChromaDB\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
from langchain.vectorstores import Chroma

langchain_chroma = Chroma(
    client=client,
    collection_name="tweets_ca",
    embedding_function=embedding_function,
)
print("There are", langchain_chroma._collection.count(), "contexts in the collection")

There are 10048 contexts in the collection


In [12]:
results = collection.query(
    # redacta una pregunta en catalan
    query_texts=["què és la independència de catalunya?"],
    n_results=5,
    include=["documents"]
)
results['documents']

[[' La independencia de Catalunya  ',
  'RT  O sigui segons  la independència de Catalunya és perfectament possible ',
  'RT  El president Quim Torra ha remarcat que la independència de Catalunya és inaturable ',
  'Ken Follet es pronuncia contra la independència de Catalunya ',
  'RT  Jo no vull la ingovernabilitat dEspanya jo vull la independència de Catalunya']]

In [14]:
title = "Positivo"

results = collection.query(
    query_texts=["què pensa la gent sobre Puidgdemont?"],
    n_results=10,
    where={"title": title}
)
results['documents']

[['RT  Carles Puigdemont  Els que ens voldrien de genolls demà tindran una resposta  ',
  '    Deja la droga hombre Què trist veure aquest babejant de ganes de detenir Puigdemont   ',
  'RT  Ciutadans desembarca aquest dissabte al poble de Puigdemont  ',
  'RT  Lestel del vespre  infantarà la nova  llum daquest poble   ',
  'RT  Puigdemont viatja a Suïssa per participar a la Festa de la República del cantó de Jura ',
  'EN DIRECTE Nieto ‘Vam dir a Puigdemont que prevalia la llei per sobre de la convivència’ ',
  'RT  Puigdemont  assegura que està disposat a mantenir una  entrevista cordial  amb  aquest diumenge htt…',
  'RT  Puigdemont va al Parlament Europeu per inaugurar una exposició sobre el català   ',
  'Artadi assumeix la interlocució amb Madrid per ordre de Puigdemont ',
  'RT    Puigdemont  Qui aturarà que això no succeeixi en països on els dirigents autoritaris estan pujant”']]

### Versión SBert + Fine-Tuning

In [8]:
# Cargar el modelo,  el parametro device='cuda' sirve para usar la GPU, pero no tenemos acceso
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [46]:
# Función de búsqueda semántica
def semantic_search(query, df, column='Texto_preprocesado', top_n=5):
    '''
    la funcion semantic_search toma una consulta y un dataframe y devuelve los resultados más similares en el dataframe
    parámetros:
    - query: texto de la consulta
    - df: dataframe con los textos a comparar
    - column: nombre de la columna que contiene los textos
    - top_n: número de resultados a devolver
    retorna:
    - dataframe con los resultados más similares
    '''
    # Codifica el texto de la consulta
    query_embedding = model.encode([query], convert_to_tensor=True).cpu()
    # Codifica los textos del dataframe
    text_embeddings = model.encode(df[column].tolist(), convert_to_tensor=True).cpu()
    # Calcula las puntuaciones de similitud
    cos_scores = util.pytorch_cos_sim(query_embedding, text_embeddings)[0]
    # Obtiene los índices de los resultados más altos
    top_results = torch.topk(cos_scores, k=top_n)
    return df.iloc[top_results.indices].assign(similarity=top_results.values)

In [47]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre els indepes??"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.5841
   jo no he dit mai que tots els indepes siguin aixi no ho penso pero lo de quimtorra no va ser nomes un article i  en el darrer any estic veien paraules como  colono  etcetc q em fan por i llastima i ja se que daltre costat nhi ha gent igual pero aixo no es excusa
Similarity Score: 0.5158
una pregunta buch es entorn puigdemont sol es entorn junqueras mirem la seva capacitat dempatia per jutjar els seus liders   jo diria que no gaire
Similarity Score: 0.5140
!!diu ladvocacia de lestat que no sha vulnerat el dret a la presumpcio dinnocencia dels presos politics   potser al far west no  pero en qualsevol estat democratic del sxxi si  jutgeu vosaltres mateixos     
Similarity Score: 0.5129
  quin fastic de gent
Similarity Score: 0.5093
total que hom diria que ja han decidit que elisenda almany fa nosa a tot arreu sembla que molta gent fa nosa per dir o pensar segons que quines coses la politica no se qui queda amb un grau de puresa al  jo segur que no


In [48]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
2592,jo no he dit mai que tots els indepes sigui...,Negativo
1926,una pregunta buch es entorn puigdemont sol es ...,Positivo
8576,!!diu ladvocacia de lestat que no sha vulnerat...,Neutral
1331,quin fastic de gent,Negativo
2411,total que hom diria que ja han decidit que eli...,Positivo


In [49]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre els feixistes??"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.7522
rt     son feixistes  fins la medul*la
Similarity Score: 0.7438
 es fan el feixisme a sobre
Similarity Score: 0.7334
el feixisme ja sha instal*lat als organs de representacio de lue aixo es gravissim  no afecta nomes catalunya es el feixisme que torna contra tots a cara descoberta protestem 
Similarity Score: 0.7319
rt  al feixisme no se'l discuteix se'l destrueix buenaventura durruti  
Similarity Score: 0.7040
rt  pero els feixistes son els altres ell no que es catala 


In [50]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
5483,rt son feixistes fins la medul*la,Neutral
2969,es fan el feixisme a sobre,Negativo
6511,el feixisme ja sha instal*lat als organs de re...,Positivo
4053,rt al feixisme no se'l discuteix se'l destrue...,Positivo
9686,rt pero els feixistes son els altres ell no q...,Negativo


In [51]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre la república??"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.6583
rt   haver proclamat una republica no simbolica
Similarity Score: 0.6236
 perque malgrat el xivarri de twitter les republiques magiques si es poses a votacio una proposta de pla ibarretxe  o federalisme de debo la gent votaria si
Similarity Score: 0.6008
!! e n q u e s t a    r e a l!! la democracia et diu fes rt la llei no diumenge ve el borbo al mwc  !!!! com sera recordat
Similarity Score: 0.5922
rt     autodeterminacio  crida el molt descafeinat i  republica  quina 
Similarity Score: 0.5710
  !! opinio erc des del  que va ser legalitzada ens ha enganyat a tots es va declarar independentista quant no ho ha estat mai  erc es declara republicana des del dia que va neixer ara fa mes de  anys i sino us ho creieu escolteu a en  durant   


In [52]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
6815,rt haver proclamat una republica no simbolica,Negativo
5680,perque malgrat el xivarri de twitter les repu...,Positivo
5160,!! e n q u e s t a r e a l!! la democracia ...,Positivo
3110,rt autodeterminacio crida el molt descafe...,Positivo
884,!! opinio erc des del que va ser legalitzad...,Positivo


In [53]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre carles Puigdemont??"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.6877
quina xuleria i quin supremacisme el sr jose antonio nieto al  de debo aquest es el nivell de la politica espanyola i els seus carrecs despres algu s'estrenyara que espany vagi com va pero tot i les ficades de pots que fan tot est escrit 
Similarity Score: 0.6852
avui  ha comentat lactualitat politica des de la seu de barcelona  tho expliquem  
Similarity Score: 0.6559
al seu prego sabina ha criticat quimtorra puigdemont i artadi aquesta opinadora organica evidentment fa veure que ha criticat tots els catalans i aixi tot 
Similarity Score: 0.6457
 seguim en directe la declaracio del vicepresident pere aragones des de madrid     
Similarity Score: 0.6423
si no hi ha cap sorpresa de darrera hora albert rivera ungira ines arrimadas com a candidata de ciutadans a les eleccions espanyoles la cap de l'oposicio fa el salt a la politica espanyola  lultima de   


In [54]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
9736,quina xuleria i quin supremacisme el sr jose a...,Positivo
334,avui ha comentat lactualitat politica des de ...,Negativo
362,al seu prego sabina ha criticat quimtorra puig...,Negativo
5245,seguim en directe la declaracio del vicepresi...,Neutral
2207,si no hi ha cap sorpresa de darrera hora alber...,Positivo


In [55]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre la independència de Catalunya??"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.7831
rt  catalansvoteu a les eleccions espanyoles que aixi aconseguireu la independencia 
Similarity Score: 0.7810
rt   si para ser catalana y votar independencia no esta mal 
Similarity Score: 0.7802
rt    en un referendum sobre la pertinenca de catalunya dins d'europa que hi votarieu
Similarity Score: 0.7682
turull  molta gent que defensa el referendum esta en contra de la independencia de catalunya mes del  dels ciutadans acceptaria el resultat de la votacio    
Similarity Score: 0.7673
 catalunya nou estat d'europa  via catalana per la independencia  n votarem n guanyarem  via lliure a la republica  a punt  la diada del si  fem la republica catalana


In [56]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
7195,rt catalansvoteu a les eleccions espanyoles q...,Positivo
4618,rt si para ser catalana y votar independenci...,Positivo
7788,rt en un referendum sobre la pertinenca de ...,Positivo
263,turull molta gent que defensa el referendum e...,Neutral
2429,catalunya nou estat d'europa via catalana pe...,Negativo


In [57]:
# Codifica el texto de la consulta
query_text = "¿quina opinió hi ha sobre el referèndum?"
results = semantic_search(query_text, df)
for _, row in results.iterrows():
    print(f"Similarity Score: {row['similarity']:.4f}")
    print(row['Texto_preprocesado'])
    print("="*50)

Similarity Score: 0.7916
    parlaven de fer un referendum al programa on es el referendum
Similarity Score: 0.7844
rt  gracies lluis per fer possible el referendum  
Similarity Score: 0.7828
rt  no es conya  l'associacio d'afectats per l'o considera una 'decepcio' que es proposi un altre referendum 
Similarity Score: 0.7739
 el senyor junqueras ara diu que el referendum anaba al programa de jxsi
Similarity Score: 0.7654
video   recorda i reviu el referendum de lo de  


In [58]:
results[['Texto_preprocesado', 'Sentimiento']]

,Texto_preprocesado,Sentimiento
6605,parlaven de fer un referendum al programa ...,Positivo
7543,rt gracies lluis per fer possible el referend...,Positivo
4097,rt no es conya l'associacio d'afectats per l...,Negativo
3587,el senyor junqueras ara diu que el referendum...,Positivo
7654,video recorda i reviu el referendum de lo de,Positivo
